In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import pickle
import os
from sklearn.cluster import KMeans
from pandas.api.types import CategoricalDtype
from sklearn.preprocessing import StandardScaler

In [ ]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [ ]:
print(np.__version__)
print(pd.__version__)

# Settings

In [ ]:
np.random.seed(42) # generating random see
tf.random.set_seed(42) # setting random seed

In [ ]:
folder ='/Users/Home/Downloads/'

# Load Data

In [ ]:
outfile = open(folder+'RNN_Train_0Valid/trainModelInput_4.pkl','rb')
valid_forTrainGen = pickle.load(outfile)
outfile.close()

# Model Config

In [ ]:
len_listOfGroups = 393651  #29104
maxFileCount = 7
input_size = 204
print(input_size)

In [ ]:

rnn_batch_size=1
steps_per_epoch_train=50000
#activation_function = 'tanh'
#rnn_initializer = 'glorot_normal'
activation_function = 'relu'
rnn_initializer = 'he_normal'


rnn_epochs=400
#rnn_loss = tf.losses.MeanSquaredError()
#rnn_loss = 'binary_crossentropy'
rnn_loss = tf.keras.losses.BinaryCrossentropy()
rnn_learning_rate='auto'
rnn_learning_rate_default = .001
#rnn_optimizer = keras.optimizers.Adam(lr=rnn_learning_rate_default, beta_1=.9, beta_2=.999 )
rnn_optimizer = keras.optimizers.Nadam(lr=rnn_learning_rate_default, beta_1=.9, beta_2=.999 )
regularizer_factor = .0000001
rnn_regularizer = keras.regularizers.l1(regularizer_factor)
dropout_rate = 0.05

# Callback Functions

In [ ]:
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
print(run_logdir)

In [ ]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
checkpoint_cb_best = keras.callbacks.ModelCheckpoint(folderOfModel+"best_model.h5", save_best_only=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(folderOfModel+"latest_model.h5", save_freq='epoch')

# Generators

In [ ]:
def nextFileData():
    while True:
#        with open('Config.txt', 'r') as f:
#            configData = json.load(f)
#            rngOfFileCounts = configData['FileCountList']
        for count in [0,1,2,3,5,6,7]:
            outfile = open(folder+'RNN_Train_0Valid/trainModelInput_'+str(count)+'.pkl','rb')
            data = pickle.load(outfile)
            outfile.close()
            print(len(data))
#            print('--'+str(count)+'--')
            np.random.shuffle(data)
            yield data
        
getNextFileData = nextFileData()

In [ ]:
def genInputData1():
    while True:
        data = next(getNextFileData)
        for g in data:
            yield np.expand_dims(g[0], axis=0) ,g[1]

In [ ]:
def genTestData(data):
    while True:
        for g in data:
            yield np.expand_dims(g[0], axis=0) ,g[1]

In [ ]:
trainGen = genInputData1()
validGen = genTestData(valid_forTrainGen)

# Model

In [ ]:
model = keras.models.Sequential([
    keras.layers.LSTM(10,return_sequences=False, input_shape=[None, input_size], recurrent_dropout =dropout_rate),
    keras.layers.Dense(1, activation= "sigmoid")
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss=rnn_loss, optimizer=rnn_optimizer, metrics=[tf.keras.metrics.AUC()])

In [ ]:
history = model.fit(x =trainGen, epochs=rnn_epochs, steps_per_epoch= steps_per_epoch_train,
                     validation_data = validGen, validation_steps = len(valid_forTrainGen),
                     callbacks = [tensorboard_cb,checkpoint_cb,checkpoint_cb_best])